In [1]:
#TODO

"""
1. predict the unlabeled data (Naive Bayes, SVM)
2. evaluate the accuracy,f1,ruc,etc?? (check with the doc later)
3. Topic Modeling
4. wordcloud(maybe,if applicable)
5. visualization(maybe, if applicable)
6. think about some other fun questions about this data set
7. paper 
    Exploring the Use of Text Classification in the Legal Domain - arXiv
    https://arxiv.org/pdf/1710.09306
    
"""

'\n1. predict the unlabeled data (Naive Bayes, SVM)\n2. evaluate the accuracy,f1,ruc,etc?? (check with the doc later)\n3. Topic Modeling\n4. wordcloud(maybe,if applicable)\n5. visualization(maybe, if applicable)\n6. think about some other fun questions about this data set\n7. paper \n    Exploring the Use of Text Classification in the Legal Domain - arXiv\n    https://arxiv.org/pdf/1710.09306\n    \n'

In [2]:
import pandas as pd
import string 
import nltk
import re 
import numpy as np
#settings
pd.options.display.max_rows=10

In [3]:
mapping = pd.read_csv('data/Interview_Mapping.csv')
mapping.head()

,Judgements,Area.of.Law
0,LNIND_1988_CAL_114,To be Tested
1,LNIND_1956_CAL_163,To be Tested
2,LNIND_1976_CAL_277,To be Tested
3,LNIND_1980_CAL_52,To be Tested
4,LNIND_1955_CAL_124,To be Tested


In [4]:
# create labels
unlabeled = []
labeled = []
labels = []

for index,row in mapping.iterrows():
    if row['Area.of.Law'] == 'To be Tested':
        unlabeled.append(row['Judgements'])
    else: 
        labeled.append(row['Judgements'])
        labels.append(row['Area.of.Law'])
        
# how much unique area of law        
len(set(labels))

41

In [5]:
# load files
import os

unlabeled_text=[]
labeled_text=[]

for name in unlabeled:
    path = os.path.join('data/',name+'.txt')
    with open(path,'r',errors = 'ignore') as f:
        unlabeled_text.append(f.read())
for name in labeled:
    path = os.path.join('data/',name+'.txt')
    with open(path,'r',errors = 'ignore') as f:
        labeled_text.append(f.read())


In [22]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

stop = stopwords.words('english')
punct = string.punctuation

labeled_cleaned=[]
unlabeled_cleaned = []

for passage in labeled_text:
        
    # remove links 
    #passage= re.sub(r'http(s)?:\/\/\S*', "", str(passage))
    #passage = ''.join([elem if elem.isalnum() or elem.isspace() else " " for elem in passage ])
    # remove \n
    passage = ''.join([elem.strip('\n') for elem in passage])
    # normalization and remove stopwords
    passage = ' '.join([elem for elem in passage.lower().split() if elem not in stop])
    #remove punctuation 
    passage = ''.join([elem.replace('[^\w\s]','') for elem in passage])
    #remove digits
    #passage = ''.join([elem for elem in passage if not elem.isdigit()])
    
    # ??? common word
    #freq = pd.Series(passage.split()).value_counts()[:10]
    
    #lemmatization
    lemmatizer = WordNetLemmatizer()
    passage = ' '.join(lemmatizer.lemmatize(elem) for elem in passage.split())
    #passage = passage.lower().split()
    labeled_cleaned.append(passage)

    
for passage in unlabeled_text:
        
    # remove links 
   # passage= re.sub(r'http(s)?:\/\/\S*', "", str(passage))
  #  passage = ''.join([elem if elem.isalnum() or elem.isspace() else " " for elem in passage ])
    # remove \n
    passage = ''.join([elem.strip('\n') for elem in passage])
    # normalization and remove stopwords
    passage = ' '.join([elem for elem in passage.lower().split() if elem not in stop])
    #remove punctuation 
    passage = ''.join([elem.replace('[^\w\s]','') for elem in passage])
    #remove digits
    #passage = ''.join([elem for elem in passage if not elem.isdigit()])
    
    # ??? common word
    #freq = pd.Series(passage.split()).value_counts()[:10]
    
    #lemmatization
    lemmatizer = WordNetLemmatizer()
    passage = ' '.join(lemmatizer.lemmatize(elem) for elem in passage.split())
    #passage = passage.lower().split()
    unlabeled_cleaned.append(passage)
    # it has to be a string so it could be processed later 


In [34]:
# split the labeled data into training and validation set 
# use 7-3 
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

train_text,val_text,train_labels,val_labels =train_test_split(labeled_cleaned,labels,test_size = 0.3,random_state = 0)

# do tfidf to get X_train and X_val
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_text)
X_val = vectorizer.transform(val_text)

# do label encoding to get y_train and y_val
encoder = LabelEncoder()
encoder.fit(labels)
y_train = encoder.transform(train_labels)
y_val = encoder.transform(val_labels)
all_labels = encoder.classes_

# do tfidf to get X_test (unlabeled text that needed to be predicted)
X_test = vectorizer.transform(unlabeled_cleaned) #transform on test set, not fit_transform

f1 score:  0.5592592592592592
[34  6  6 14 14  6  6 28 23  6 23  6 23 39  6 14  6 39 37  6 23 23 37  6
 39  6 23  6 39 23  6 39  6 23 34 14  1  6 14 15  6  6 13 23  6 34 14  6
 39 14 13 39 39  6  1 28  1 23  6  6 23  6 39 13 34  6  6  6  6 14  6 23
  6  6 37  6 23 14  6 14  6  7  6 39  6 39 34 37 23 23 39 23 23 14  6  6
 23 38 13  6]
Train accuracy :  0.7313195548489666
Test accuracy :   0.5592592592592592


In [47]:
# use Naive Bayes
from sklearn import naive_bayes
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

modelNB = naive_bayes.MultinomialNB()
modelNB.fit(X_train,y_train)
predicted_labels = modelNB.predict(X_val)
print('f1 score: ',f1_score(y_val,predicted_labels,average='micro'))
result = modelNB.predict(X_test)
print(result)

yHat_train = modelNB.predict(X_train)
yHat_val = modelNB.predict(X_val)

print("Train accuracy : ", accuracy_score(y_train, yHat_train))
print("Test accuracy :  ", accuracy_score(y_val, yHat_val))

f1 score:  0.31851851851851853
[ 6  6  6  6  6  6  6  6 23  6 23  6 23 39  6  6  6 39  6  6 23 23  6  6
 39  6 23  6 39 23  6 39  6 23  6 14  6  6  6  6  6  6  6 23  6  6  6  6
 39  6  6 39 39  6  6  6  6  6  6  6 23  6  6  6  6  6  6  6  6  6  6 23
  6  6  6  6 23  6  6  6  6  6  6 39  6  6  6  6  6 23 39 23 23  6  6  6
 23  6  6  6]
Train accuracy :  0.36089030206677264
Test accuracy :   0.31851851851851853


In [45]:
# use SVM
from sklearn import svm

modelSVM = svm.SVC(kernel='linear',C=0.1)
modelSVM.fit(X_train,y_train)
predicted_labels = modelSVM.predict(X_val)
print('f1 score: ',f1_score(y_val,predicted_labels,average='micro'))
result = modelSVM.predict(X_test)
print(result)

yHat_train = modelSVM.predict(X_train)
yHat_val = modelSVM.predict(X_val)

print("Train accuracy : ", accuracy_score(y_train, yHat_train))
print("Test accuracy :  ", accuracy_score(y_val, yHat_val))


f1 score:  0.2814814814814815
[ 6  6  6  6  6  6  6  6 23  6  6  6 23  6  6  6  6  6  6  6 23 23  6  6
  6  6 23  6  6  6  6  6  6 23  6  6  6  6  6  6  6  6  6 23  6  6  6  6
  6  6  6  6 39  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6 23  6  6  6  6  6  6  6  6  6  6  6  6 23  6 23 23  6  6  6
 23  6  6  6]
Train accuracy :  0.26232114467408585
Test accuracy :   0.2814814814814815


In [48]:
# use Linear Regression
from sklearn.linear_model import LogisticRegression


modelLR = LogisticRegression()
modelLR.fit(X_train,y_train)
predicted_labels = modelLR.predict(X_val)
print('f1 score: ',f1_score(y_val,predicted_labels,average='micro'))
result = modelLR.predict(X_test)
print(result)
yHat_train = modelLR.predict(X_train)
yHat_val = modelLR.predict(X_val)

print("Train accuracy : ", accuracy_score(y_train, yHat_train))
print("Test accuracy :  ", accuracy_score(y_val, yHat_val))

f1 score:  0.5592592592592592
[34  6  6 14 14  6  6 28 23  6 23  6 23 39  6 14  6 39 37  6 23 23 37  6
 39  6 23  6 39 23  6 39  6 23 34 14  1  6 14 15  6  6 13 23  6 34 14  6
 39 14 13 39 39  6  1 28  1 23  6  6 23  6 39 13 34  6  6  6  6 14  6 23
  6  6 37  6 23 14  6 14  6  7  6 39  6 39 34 37 23 23 39 23 23 14  6  6
 23 38 13  6]
Train accuracy :  0.7313195548489666
Test accuracy :   0.5592592592592592


In [79]:
a = all_labels[result]
a


array(['Property Laws', 'Civil Procedure', 'Civil Procedure',
       'Criminal Procedure', 'Criminal Procedure', 'Civil Procedure',
       'Civil Procedure', 'Local Government', 'Income Tax',
       'Civil Procedure', 'Income Tax', 'Civil Procedure', 'Income Tax',
       'Tenancy Laws', 'Civil Procedure', 'Criminal Procedure',
       'Civil Procedure', 'Tenancy Laws', 'Service Law',
       'Civil Procedure', 'Income Tax', 'Income Tax', 'Service Law',
       'Civil Procedure', 'Tenancy Laws', 'Civil Procedure', 'Income Tax',
       'Civil Procedure', 'Tenancy Laws', 'Income Tax', 'Civil Procedure',
       'Tenancy Laws', 'Civil Procedure', 'Income Tax', 'Property Laws',
       'Criminal Procedure', 'Alternative Dispute Resolution',
       'Civil Procedure', 'Criminal Procedure', 'Customs',
       'Civil Procedure', 'Civil Procedure', 'Criminal Laws',
       'Income Tax', 'Civil Procedure', 'Property Laws',
       'Criminal Procedure', 'Civil Procedure', 'Tenancy Laws',
       'Criminal 

In [109]:
# write result in csv
with open('predictions.csv','w') as f:
    f.write('Judgements' + '\t' + 'Area of Law' + '\n')
    predictionList = all_labels[result]
    for i in range(0, len(result)):
        f.write(unlabeled[i] + '\t' + predictionList[i] + '\n')
        


In [117]:
sss = pd.read_csv('predictions.csv')
print(sss)

                     Judgements\tArea of Law
0        LNIND_1988_CAL_114\tProperty Laws\t
1      LNIND_1956_CAL_163\tCivil Procedure\t
2      LNIND_1976_CAL_277\tCivil Procedure\t
3    LNIND_1980_CAL_52\tCriminal Procedure\t
4   LNIND_1955_CAL_124\tCriminal Procedure\t
..                                       ...
95     LNIND_1980_CAL_279\tCivil Procedure\t
96          LNIND_1980_CAL_229\tIncome Tax\t
97     LNIND_1988_CAL_232\tSuccession Laws\t
98       LNIND_1957_CAL_142\tCriminal Laws\t
99     LNIND_1988_CAL_107\tCivil Procedure\t

[100 rows x 1 columns]


In [116]:
a = '\t'+'aaa\t'+'bbb' +'\t'
print(a)

	aaa	bbb	
